In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, Model, regularizers
from tensorflow.keras.layers import Conv2D, Dense, Input, Flatten, MaxPooling2D, BatchNormalization, Dropout

from tensorflow.keras.datasets import cifar10

In [36]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape)
print(y_train.shape)

x_train = keras.utils.normalize(x_train, axis = 1)
x_test = keras.utils.normalize(x_test, axis = 1)

(50000, 32, 32, 3)
(50000, 1)


In [24]:
vgg_16 = keras.applications.vgg16.VGG16(
    weights = 'imagenet',
    include_top = False, 
    classes = 10,
    input_shape = (32, 32, 3)
)

58889256/58889256 [==============================] - 2s 0us/step


In [25]:
vgg_16.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

### Sequential

In [38]:
model = Sequential()
for layer in vgg_16.layers[:-1]:
    model.add(layer)
    
for layer in model.layers:
    layer.trainable = False

model.add(Flatten())
model.add(Dense(512, activation = 'relu', name = 'hidden1'))
model.add(Dropout(0.4))
model.add(Dense(256, activation = 'relu', name = 'hidden2'))
model.add(Dropout(0.4))
model.add(Dense(10, activation = 'softmax', name = 'predictions'))

In [39]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0         
                                                                 
 block3_conv1 (Conv2D)       (None, 8, 8, 256)        

In [40]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(),
    optimizer = keras.optimizers.Adam(learning_rate = 1e-4), # keras.optimizers.SGD(lr = 0.001, momentum =  0.9)
    metrics = ['accuracy']
)

model.fit(x_train, y_train, validation_split = 0.1, shuffle = True, batch_size = 64, epochs = 15)

Epoch 1/15
704/704 [==============================] - 81s 114ms/step - loss: 2.0060 - accuracy: 0.2742 - val_loss: 1.7226 - val_accuracy: 0.4128
Epoch 2/15
704/704 [==============================] - 81s 114ms/step - loss: 1.7489 - accuracy: 0.3779 - val_loss: 1.6144 - val_accuracy: 0.4294
Epoch 3/15
704/704 [==============================] - 83s 117ms/step - loss: 1.6548 - accuracy: 0.4140 - val_loss: 1.5330 - val_accuracy: 0.4666
Epoch 4/15
704/704 [==============================] - 85s 120ms/step - loss: 1.5963 - accuracy: 0.4394 - val_loss: 1.5007 - val_accuracy: 0.4710
Epoch 5/15
704/704 [==============================] - 82s 116ms/step - loss: 1.5572 - accuracy: 0.4484 - val_loss: 1.4575 - val_accuracy: 0.4884
Epoch 6/15
704/704 [==============================] - 81s 116ms/step - loss: 1.5255 - accuracy: 0.4635 - val_loss: 1.4464 - val_accuracy: 0.4922
Epoch 7/15
704/704 [==============================] - 83s 117ms/step - loss: 1.4962 - accuracy: 0.4730 - val_loss: 1.4189 - val_ac

In [41]:
model.evaluate(x_test, y_test, batch_size = 64)

157/157 [==============================] - 17s 106ms/step - loss: 1.3618 - accuracy: 0.5202


[1.3618457317352295, 0.5202000141143799]

### Functional

In [59]:
vgg_16 = keras.applications.vgg16.VGG16(
    weights = 'imagenet',
    include_top = False,
    classes = 10,
    input_shape = (32, 32, 3)
)

In [60]:
x = vgg_16.layers[-1].output
x = Flatten()(x)
x = Dense(512, activation = 'relu')(x)
x = Dropout(0.4)(x)
x = Dense(256, activation = 'relu')(x)
x = Dropout(0.4)(x)
output = Dense(10, activation = 'softmax')(x)

model = Model(inputs = vgg_16.input, outputs = output)

In [61]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(),
    optimizer = keras.optimizers.SGD(learning_rate = 0.001, momentum =  0.9),
    metrics = ['accuracy']
)

model.fit(x_train, y_train, validation_split = 0.1, shuffle = True, batch_size = 64, epochs = 15)

Epoch 1/15
704/704 [==============================] - 367s 519ms/step - loss: 1.6002 - accuracy: 0.4260 - val_loss: 1.0373 - val_accuracy: 0.6468
Epoch 2/15
704/704 [==============================] - 373s 529ms/step - loss: 0.9923 - accuracy: 0.6646 - val_loss: 0.8029 - val_accuracy: 0.7246
Epoch 3/15
704/704 [==============================] - 372s 529ms/step - loss: 0.7930 - accuracy: 0.7365 - val_loss: 0.7185 - val_accuracy: 0.7558
Epoch 4/15
704/704 [==============================] - 373s 530ms/step - loss: 0.6610 - accuracy: 0.7810 - val_loss: 0.6731 - val_accuracy: 0.7690
Epoch 5/15
704/704 [==============================] - 364s 517ms/step - loss: 0.5637 - accuracy: 0.8142 - val_loss: 0.5813 - val_accuracy: 0.7988
Epoch 6/15
704/704 [==============================] - 373s 530ms/step - loss: 0.4962 - accuracy: 0.8352 - val_loss: 0.5598 - val_accuracy: 0.8166
Epoch 7/15
704/704 [==============================] - 381s 541ms/step - loss: 0.4361 - accuracy: 0.8570 - val_loss: 0.5789 -

In [62]:
model.evaluate(x_test, y_test, batch_size = 64)

157/157 [==============================] - 18s 112ms/step - loss: 0.7174 - accuracy: 0.8316


[0.7173755764961243, 0.83160001039505]